## Word2Vec, ELMo, Doc2Vec의 이해

### Word2Vec
단어를 원핫 벡터로 표현하는 것은 기계적이지만, 이 원핫 벡터를 밀집 벡터로 변환하려면 반드시 **학습**이 있어야 한다.
Word2Vec 기법에서 학습의 목적은 단어에 의미적인 정보를 함축함으로써 유사도를 계산하거나 연산을 수행하고, 더 나아가 학습된 결과를 다른 작업에서도 수행할 수 있도록 **전이학습**을 지원한다.
Word2Vec은 주변의 단어를 이용해 중심에 있는 단어를 예측하도록 학습한다. (CBow)
Word2Vec은 **다음 단어 혹은 주변 단어에 대한 예측을 잘할 수 있도록 학습함**으로써 문맥을 이해시키고, 밀집 벡터에 그러한 문맥 정보를 담으려는 시도이다.

WordVec은 CBOW와 Skip-Gram의 두 가지 학습 방식을 갖고 있다.
CBOW : 주변의 단어를 이용해 중심단어를 예측하도록 학습을 수행한다.
Skip-Gram : 중심의 한 단어를 이용해 주변의 단어를 예측한다.

window : 앞뒤 단어들을 몇 개씩 예측에 사용할지 결정하는 범위  (CBow에서 사용)


In [1]:
##학습된 word2vec 모형 가져오기
import gensim.downloader as api
wv = api.load('glove-wiki-gigaword-50')
type(wv)

[==================================================] 100.0% 66.0/66.0MB downloaded


gensim.models.keyedvectors.KeyedVectors

In [2]:
vec_king = wv['king']
print("#size of vector : ", len(vec_king)) #50개의  밀집 벡터 로드
print("#vector of king : ", vec_king) # 밀집 벡터 안의 내용

#size of vector :  50
#vector of king :  [ 0.50451   0.68607  -0.59517  -0.022801  0.60046  -0.13498  -0.08813
  0.47377  -0.61798  -0.31012  -0.076666  1.493    -0.034189 -0.98173
  0.68229   0.81722  -0.51874  -0.31503  -0.55809   0.66421   0.1961
 -0.13495  -0.11476  -0.30344   0.41177  -2.223    -1.0756   -1.0783
 -0.34354   0.33505   1.9927   -0.04234  -0.64319   0.71125   0.49159
  0.16754   0.34344  -0.25663  -0.8523    0.1661    0.40102   1.1685
 -1.0137   -0.21585  -0.15155   0.78321  -0.91241  -1.6106   -0.64426
 -0.51042 ]


In [3]:
#유사도는 cosine similarity를 기본으로 사용함
print(wv.similarity('king', 'man'), 'vs', wv.similarity('king', 'woman'))
print(wv.similarity('queen', 'man'), 'vs', wv.similarity('queen', 'woman'))
print("미니밴에 가까운 차 : ", wv.most_similar(positive=['car', 'minivan'], topn=3))
print("여성, 왕에는 가까우면서 남성과는 먼 단어 :", wv.most_similar(positive=['queen', 'king'], negative=['man'], topn = 1))
print("breakfast cereal dinner lunch 중 다른 단어들과 거리가 가장 먼 단어 : ", wv.doesnt_match("breakfast cereal dinner lunch".split()))

0.53093773 vs 0.41133782
0.5366701 vs 0.6003106
미니밴에 가까운 차 :  [('truck', 0.9100274443626404), ('suv', 0.904007613658905), ('jeep', 0.8619830012321472)]
여성, 왕에는 가까우면서 남성과는 먼 단어 : [('coronation', 0.7994614243507385)]
breakfast cereal dinner lunch 중 다른 단어들과 거리가 가장 먼 단어 :  cereal


In [5]:
#거리 반환
print("distance between cat and dog : {:.2f}".format(wv.distance('cat', 'dog')))
print("{:.4f}".format((wv.n_similarity(['bulgogi', 'shop'], ['japanese', 'restaurant']))))
print("{:.4f}".format((wv.n_similarity(['bulgogi', 'shop'], ['korean', 'restaurant']))))
print("{:.4f}".format((wv.n_similarity(['bulgogi', 'shop'], ['french', 'restaurant']))))

distance between cat and dog : 0.08
0.5375
0.5627
0.4377


#### 결과
최근에는 BERT를 활용한 전이학습을 주로 사용하므로 Word2Vec의 사용은 점차 줄어드는 추세에 있다.

#### 다른 방법

1 . GloVe
word2vec이 주변단어를 중심으로 학습되어 문서 전체에서 다른 단어들과의 관계가 잘 반영되지 않는 다는 점을 극복하기 위한 방법으로 제시했다.
2 . FastText
word2vec이 학습 문서에 없는 단어에 대해 취약하다는 것이다. 워드 임베딩 과정에서 없는 단어를 만나면 OOV 토큰을 부여하는 방식으로 처리한다.
이렇게 되면, 다른 단어와 **유사도** 계산이 불가능하다.
Fast-Text는 단어에 문자 단위의 N-gram을 적용하여 이를 해결하고자 하였다.
예를 들어, Hello를 bi-gram으로 적용하면 He/el/ll/lo와 같이 토큰을 만든다.
특히 기존의 단어를 변형하거나 결합해서 만든 새로운 단어에 대해 뛰어난 유사도를 보여준다.

### ELMo

word2vec의 문제점
1 . 동음이의어
word2vec에서는 동음이의어가 동일한 벡터로 임베딩된다. 이러한 단어가 서로 다른 벡터로 임베딩할 수 있다면, 원래 추구했던 문맥의 파악이
더 잘될 것이다.
→ 이러한 문제점을 해결한 방식이 ELMo이다.

ELMo는 임벡딩 벡터가 **고정되어 있지 않다.**
즉, 전이학습을 위해 word2vec은 고정된 벡터를 가져다 쓰는 반면, ELMo는 학습된 모형을 가져와 주어진 문장에 맞게 **가변적인 임베딩 벡터**를 생성한다.

언어모델은 문장 혹은 단어의 시퀀스에 대해 확률을 할당함으로써 그 문장이 얼마나 자연스러운지를 알 수 있게 해준다.
예를 들어,
나는 배가 고파서 밥을 ____.
→ "먹었다"를 예측할 수 있는데 이는 문맥에 대한 이해가 있어야 가능하므로, 학습을 통해 다음 단어를 잘 예측하도록 만들면 문맥에 대한 이해가 높아졌다고 볼 수 있다.

ELMo는 사전학습된 양방향 LSTM을 사용해 임베딩을 수행한다.
양방향 LSTM은 앞 단어들이 뒤에 미치는 영향을 더해, **뒤 단어에서 앞 단어 방향으로도 LSTM 층을 추가해** 반대 방향의 영향도 학습한다.
(앞/ 뒤 영향 모두 파악)

**ELMo의 과정**
문장의 단어들은 먼저 일반적인 임베딩을 통해 벡터로 변환되고, 이 임베딩 벡터가 첫번째 양방향 LSTM 층의 입력이 된다.
동일한 임베딩 벡터가 정방향 LSTM과 역방향 LSTM에 입력으로 들어가 각각의 출력을 만들어내고, 이 둘을 합쳐 첫째 층의 출력이 된다.
이 때, 정방향과 역방향 LSTM은 서로 합쳐지지 않고 **각각의 입력*으로 사용된다. (독립적 학습)

그 뒤, 정방향 LSTM과 역방향 LSTM 결과를 붙여 하나의 벡터로 만든다.
그 후, 각 층의 벡터에 가중치를 곱한다. →  각 층의 결과 중에서 어떤 것이 최종 결과에 더 많은 영향을 미치도록 할지 결정하는 것으로 학습을 통해 조정될 수 있다.
입력층을 포함한 모든 층의 결과를 더해 최종 임베딩 벡터를 만든다.

ELMo는 모형 자체만을 전이하고 주어진 문장을 모형에 적용한다.
ELMo는 BERT의 주요한 기반이다. (하지만, 지금은 사용 안함)

-tensorflow hub 필요

※ 이제는 활용도가 떨어져... 실습 안 함...

### Doc2Vec

대부분 단어에 대한 임베딩을 하는 반면, Doc2Vec은 **문서에 대해 직접 임베딩한다**는 점에서 큰 차이가 있다.
Doc2Vec에서는 문서의 ID를 단어와 동일하게 취급해서 학습과정에 포함시킨다.
문서에 나온 여러 단어들이 문맥맥정보가 반영되어 학습되는 동안, 문서 ID도 그 문서에 나온 단어들의 문맥정보를 같이 학습한다.

**학습 구조**
1 . DM(Distributed Memory)
Word2Vec의 CBOW에 문서 ID를 추가한 형태의 학습
앞의 단어들과 문서 ID를 이용해 다음 단어를 예측
2 . DBOW
skip-gram에 문서 ID를 추가한 형태의 학습
문서 ID로 일련의 단어들을 예측하는 방식
